In [2]:
import pandas as pd

In [3]:
# Load in dataframe
df_final = pd.read_csv("/Users/femke/Documents/Uni/DSS/Thesis/Data/ml1m_df_final.csv")

In [44]:
#TRY WITH THIS ONE
# Save and load 
#df_temp.to_csv('/Users/femke/Documents/Uni/DSS/Thesis/Data/df_temp_before_scaling.csv', index=False)
df_temp = pd.read_csv("/Users/femke/Documents/Uni/DSS/Thesis/Data/df_temp_before_scaling.csv")

In [45]:
df_final = df_temp.copy()

In [50]:
df_final.head()

,UserID,MovieID,Rating,Year,Month,Day,Hour,Age,Release_year,Avg_rating_user,...,Favourite_Fantasy,Favourite_Film-Noir,Favourite_Horror,Favourite_Musical,Favourite_Mystery,Favourite_Romance,Favourite_Sci-Fi,Favourite_Thriller,Favourite_War,Favourite_Western
0,1,1193,5,2000,12,31,23,0,1975,4.19,...,False,False,False,False,False,False,False,False,True,False
1,2,1193,5,2000,12,31,22,6,1975,3.71,...,False,False,False,False,False,False,False,False,False,True
2,12,1193,4,2000,12,31,0,2,1975,3.83,...,False,False,False,False,False,False,False,False,False,False
3,15,1193,4,2000,12,30,19,2,1975,3.32,...,False,False,False,False,False,False,False,False,False,False
4,17,1193,5,2000,12,30,7,5,1975,4.08,...,False,False,False,False,False,False,False,False,False,False


In [47]:
# Sample the data for increased efficiency
from sklearn.model_selection import train_test_split

# Assuming df_temp is your existing dataframe
df_sampled, _ = train_test_split(df_final, test_size=0.6, stratify=df_final[['Rating']])


# Random Forest

In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Separate features and target
X = df_sampled.drop(columns=['Rating'])
y = df_sampled['Rating']

# Split data into 60% training, 20% validation, and 20% test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp)

# Initialize and fit Random Forest Classifier
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

# Predict on validation and test sets
val_predictions = rf_model.predict(X_val)
test_predictions = rf_model.predict(X_test)

# Evaluate performance
def evaluate_performance(y_true, y_pred):
    print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
    print(f"Precision: {precision_score(y_true, y_pred, average='macro')}")
    print(f"Recall: {recall_score(y_true, y_pred, average='macro')}")
    print(f"F1 Score: {f1_score(y_true, y_pred, average='macro')}")

print("Validation Set Performance:")
evaluate_performance(y_val, val_predictions)

print("Test Set Performance:")
evaluate_performance(y_test, test_predictions)


Validation Set Performance:
Accuracy: 0.9943512003699214
Precision: 0.9931450471009262
Recall: 0.9868835810654726
F1 Score: 0.9899195179321797
Test Set Performance:
Accuracy: 0.994863591486809
Precision: 0.9937126710704506
Recall: 0.9879005562992355
F1 Score: 0.9907323459971968


oude versie  hier beneden

In [21]:
# Define the target variable and features
X = df_sampled.drop(columns=['Rating'])
y = df_sampled['Rating']

In [22]:
# Split the data into 60% training, 20% validation and 20% test

# Split 60% training and 40% test + val
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)

# Split the remainder in test (50%) and validation 950%)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)


In [23]:
# Initialize the Random Forest model
rf_baseline = RandomForestClassifier(random_state=42)

In [24]:
# Train the model
rf_baseline.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [25]:
y_pred = rf_baseline.predict(X_test)

In [26]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.9935
Precision: 0.9936
Recall: 0.9935
F1 Score: 0.9935


# Gradient boosting

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [21]:
# Assuming df_final is your dataframe and 'Rating' is the target variable
X = df_final.drop(columns=['Rating'])
y = df_final['Rating']

In [22]:
# Splitting the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# Creating the Gradient Boosting model
gb_baseline = GradientBoostingClassifier(random_state=42)
gb_baseline.fit(X_train, y_train)

GradientBoostingClassifier(random_state=42)

In [24]:
# Making predictions
y_pred = gb_baseline.predict(X_test)

In [25]:
# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.3740
Precision: 0.3846
Recall: 0.3740
F1 Score: 0.2983
